# Metadata

```
Course: DS 5001
Semester Project
Topic: Gilmore Girls
Author: Alice Bogdan
Computing id: ucn3qn
```

Purpose: Using ETA on Gilmore Girls tv scripts. Preprocess the data

In [1]:
# import requests
# from lxml import etree
# import pandas as pd
# import numpy as np
# from io import StringIO
# from glob import glob
# import re

# series_id = 22
# outdir = './dump'
# base_url = "https://transcripts.foreverdreaming.org"

# parser = etree.HTMLParser()

# def get_pages(series_id, offset=0, outdir=''):
#     src_url = f"{base_url}/viewforum.php?f={series_id}&start={offset}"
#     r = requests.get(src_url)
#     tree = etree.parse(StringIO(r.text), parser)
#     page_urls = [f"{base_url}/{path[2:]}" for path in tree.xpath('//a[@class="topictitle"]/@href')]
#     for page_url in page_urls:
#         f, t, sid = page_url.split('?')[-1].split('&')
#         page_r = requests.get(page_url)
#         out_file_name = f"f{f.split('=')[-1]}-r{t.split('=')[-1]}.html"
#         with open(f"{outdir}/{out_file_name}", 'w') as out:
#             out.write(page_r.text)

# for offset in range(0, 175, 25):
#     print(offset)
#     get_pages(22, offset, outdir)

# xpaths = {
#        'title': "//div[@class='boxheading']/h2/text()",
#         'time': "//time/@datetime",
#         'content': "//div[@class='postbody']/p/text()"
# }

# data = []
# for page in glob(f"{outdir}/*.html"):
#     tree = etree.parse(page, parser)
#     title = tree.xpath(xpaths['title'])[0]
#     time = tree.xpath(xpaths['time'])[0]
#     lines = tree.xpath(xpaths['content'])    
#     for i, line in enumerate(lines):
#         data.append((title, time, i, line))
# df = pd.DataFrame(data, columns=['title_raw','time_raw','line_num','doc_raw'])

# df.to_csv('gilmore-girls.csv')

# Set Up

In [2]:
#libraries
import pandas as pd
import numpy as np
import nltk
# from sklearn.decomposition import PCA
from scipy.linalg import norm
import plotly.express as px
import seaborn as sns; sns.set()
import re

In [3]:
#nltk.download('stopwords')

In [4]:
import os
import sys

# Load Data

In [5]:
#import data
DATA = pd.read_csv("gilmore-girls.csv")

In [6]:
#split title_raw into three columns: Season, Episode, and Title
DATA[['Season', 'Title']] = DATA['title_raw'].str.split('-', n = 1, expand = True)
DATA[['Season', 'Episode']] = DATA['Season'].str.split('x', n = 1, expand = True)

In [7]:
#remove whitespace
DATA['Episode'] = DATA['Episode'].str.strip()
DATA['Season'] = DATA['Season'].str.strip()

In [8]:
#subset data to remove Unamed: 0 and title_raw
DATA = DATA[["Season", "Episode", "Title","time_raw","line_num","doc_raw"]]

need to:
cut by scene
get rid of anything in [] or ()
capture names starts with all caps followed : 

In [9]:
#keep only seasons 1-7 (season 8 is an exception)
DATA = DATA[DATA['Season'].isin(['01', '02', '03', '04', '05', '06', '07'])]
DATA = DATA.sort_values(['Season', 'Episode'])

In [10]:
#not every episode has scenes
DATA[DATA['doc_raw'].str.contains("cut to", case = False)][0:25]

,Season,Episode,Title,time_raw,line_num,doc_raw
61833,01,01,The Pilot,2000-10-17T15:42:16Z,80,CUT TO THE INDEPENDENCE INN
61878,01,01,The Pilot,2000-10-17T15:42:16Z,125,CUT TO OUTSIDE
61897,01,01,The Pilot,2000-10-17T15:42:16Z,144,CUT TO CLASSROOM
61906,01,01,The Pilot,2000-10-17T15:42:16Z,153,CUT TO THE INDEPENDENCE INN
61943,01,01,The Pilot,2000-10-17T15:42:16Z,190,CUT TO OUTSIDE
61999,01,01,The Pilot,2000-10-17T15:42:16Z,246,CUT TO INDEPENDENCE INN KITCHEN
62059,01,01,The Pilot,2000-10-17T15:42:16Z,306,CUT TO INDEPENDENCE INN LOBBY
62069,01,01,The Pilot,2000-10-17T15:42:16Z,316,CUT TO LORELAI'S HOUSE
62072,01,01,The Pilot,2000-10-17T15:42:16Z,319,CUT TO LORELAI'S FRONT PORCH
62121,01,01,The Pilot,2000-10-17T15:42:16Z,368,CUT TO ELDER GILMORE RESIDENCE


# Preprocessing

## Remove stage directions

In [11]:
#regex to capture lines beginning and ending with [] or () indicating stage directions
stage_dir = r"(\(.*\))|(\[.*\])"

In [12]:
#COPY = DATA.copy()

In [13]:
#run regex code to find lines with stage directions
stage_lines = DATA.doc_raw.str.match(stage_dir, case=False) # Returns a truth vector

In [14]:
#remove stage direction lines
DATA = DATA.loc[-stage_lines]

In [15]:
#remove stage directions found within lines
DATA['doc_raw'] = DATA['doc_raw'].replace(regex = stage_dir, value='')

In [16]:
DATA

,Season,Episode,Title,time_raw,line_num,doc_raw
61753,01,01,The Pilot,2000-10-17T15:42:16Z,0,1.01 - Pilot
61754,01,01,The Pilot,2000-10-17T15:42:16Z,1,written by Amy Sherman-Palladino
61755,01,01,The Pilot,2000-10-17T15:42:16Z,2,directed by Lesli Linka Glatter
61756,01,01,The Pilot,2000-10-17T15:42:16Z,3,OPEN IN STARS HOLLOW
61758,01,01,The Pilot,2000-10-17T15:42:16Z,5,"LORELAI: Please, Luke. Please, please, please."
...,...,...,...,...,...,...
100866,07,22,Bon Voyage,2007-05-27T10:54:44Z,808,LORELAI: Well how will people know you're the ...
100867,07,22,Bon Voyage,2007-05-27T10:54:44Z,809,RORY: I don't know.
100868,07,22,Bon Voyage,2007-05-27T10:54:44Z,810,LORELAI: I guess they'll just have to read you...
100869,07,22,Bon Voyage,2007-05-27T10:54:44Z,811,RORY: I guess so.


## Capture Characters

In [17]:
#regex to capture characters (example, "RORY:"")
char_pat = r"^[A-Z]* ?:"

In [18]:
#find all lines that begin with character talking
char_lines = DATA.doc_raw.str.match(char_pat, case=False) # Returns a truth vector

In [19]:
#filter data for only lines where characters talk
#this also removes fluff lines such as the director, episode number, "OPEN IN", "THE END", etc
DATA = DATA.loc[char_lines]

In [20]:
DATA

,Season,Episode,Title,time_raw,line_num,doc_raw
61758,01,01,The Pilot,2000-10-17T15:42:16Z,5,"LORELAI: Please, Luke. Please, please, please."
61759,01,01,The Pilot,2000-10-17T15:42:16Z,6,LUKE: How many cups have you had this morning?
61760,01,01,The Pilot,2000-10-17T15:42:16Z,7,LORELAI: None.
61761,01,01,The Pilot,2000-10-17T15:42:16Z,8,LUKE: Plus...
61762,01,01,The Pilot,2000-10-17T15:42:16Z,9,"LORELAI: Five, but yours is better."
...,...,...,...,...,...,...
100865,07,22,Bon Voyage,2007-05-27T10:54:44Z,807,RORY: Well It wasn't very flattering.
100866,07,22,Bon Voyage,2007-05-27T10:54:44Z,808,LORELAI: Well how will people know you're the ...
100867,07,22,Bon Voyage,2007-05-27T10:54:44Z,809,RORY: I don't know.
100868,07,22,Bon Voyage,2007-05-27T10:54:44Z,810,LORELAI: I guess they'll just have to read you...


## Split doc_raw to form column for characters

In [21]:
DATA[['Character', 'lines']] = DATA['doc_raw'].str.split(':', n = 1, expand = True)

/Users/alicebogdan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [22]:
DATA = DATA[["Season","Episode","Title","time_raw","line_num","Character","lines"]]

In [23]:
DATA

,Season,Episode,Title,time_raw,line_num,Character,lines
61758,01,01,The Pilot,2000-10-17T15:42:16Z,5,LORELAI,"Please, Luke. Please, please, please."
61759,01,01,The Pilot,2000-10-17T15:42:16Z,6,LUKE,How many cups have you had this morning?
61760,01,01,The Pilot,2000-10-17T15:42:16Z,7,LORELAI,None.
61761,01,01,The Pilot,2000-10-17T15:42:16Z,8,LUKE,Plus...
61762,01,01,The Pilot,2000-10-17T15:42:16Z,9,LORELAI,"Five, but yours is better."
...,...,...,...,...,...,...,...
100865,07,22,Bon Voyage,2007-05-27T10:54:44Z,807,RORY,Well It wasn't very flattering.
100866,07,22,Bon Voyage,2007-05-27T10:54:44Z,808,LORELAI,Well how will people know you're the world's ...
100867,07,22,Bon Voyage,2007-05-27T10:54:44Z,809,RORY,I don't know.
100868,07,22,Bon Voyage,2007-05-27T10:54:44Z,810,LORELAI,I guess they'll just have to read your stuff.


In [24]:
DATA['Character'] = DATA['Character'].str.strip()
DATA['Character'] = DATA['Character'].str.upper()

<ipython-input-24-d8d10a87a192>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['Character'] = DATA['Character'].str.strip()
<ipython-input-24-d8d10a87a192>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['Character'] = DATA['Character'].str.upper()


In [25]:
DATA['Character'].sort_values().unique()

array(['', 'ADVISOR', 'AK', 'ALAN', 'ALEX', 'ALEXANDRA', 'ALISON', 'ALL',
       'ALTHEA', 'AMIR', 'ANDRE', 'ANDREA', 'ANDREW', 'ANDY', 'ANN',
       'ANNA', 'ANNABELLE', 'ANNOUNCER', 'ANTONIA', 'APRIL', 'ARLETTA',
       'ARTHUR', 'ASHER', 'ATTENDANT', 'AUBREY', 'AUCTIONEER', 'AUDREY',
       'AURORA', 'AVA', 'AYOR', 'BABETE', 'BABETTE', 'BAND', 'BARBARA',
       'BARBETTE', 'BARRY', 'BARTENDER', 'BEATRICE', 'BEAU', 'BELLBOY',
       'BEN', 'BENNY', 'BERT', 'BEVERLY', 'BIKER', 'BIKERS', 'BILL',
       'BILLY', 'BOB', 'BOBBI', 'BOBBY', 'BONNIE', 'BOOTSY', 'BOOZOO',
       'BOTH', 'BOUNCER', 'BOY', 'BOYS', 'BRAD', 'BRADLEY', 'BRAIN',
       'BRANDON', 'BRENNON', 'BRIAN', 'BRIDE', 'BRUCE', 'BRUNETTE',
       'BUDDY', 'BUTCHER', 'CAESAR', 'CAESER', 'CAITLYN', 'CARL', 'CAROL',
       'CAROLE', 'CAROLINE', 'CAROLYN', 'CARRIE', 'CASHIER', 'CATHERINE',
       'CELINE', 'CHAD', 'CHARLENE', 'CHARLESTON', 'CHARLIE', 'CHARLOTTE',
       'CHASE', 'CHEF', 'CHERRY', 'CHERYL', 'CHESTER', 'CHILDREN', 

In [26]:
#it's Rory
DATA[DATA['Character'] == ""]

,Season,Episode,Title,time_raw,line_num,Character,lines
17465,01,20,P.S. I Lo...,2001-05-15T14:32:16Z,782,,Please.


In [27]:
DATA['Episode'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22'],
      dtype=object)

In [28]:
#incorrect names lists
lorelai_sp = ['LOREAI', 'LOREALI', 'LORELA', 'LORELAIL', 'LORELI', 'LORLAI', 'LORLEAI', 'LORLELAI', 'L','ORELAI']
luke_sp = ['LUK']
rory_sp = ['R',"",'ORY']
richard_sp = ['RICHRAD','GRANDPA']
emily_sp = ['EMIL','GRANDMOTHER','GRANDMA','RMILY']
sookie_sp = ['SOOKEI', 'SOOKI', 'SOOKIES']
caesar_sp = ['CAESER']
christopher_sp = ['CHRISTOHPER','CHRISTOPER','CHRSTOPHER','CHRISTOPER','CHRIS']
lindsay_sp = ['LINDAY'] #dean's wife
headmaster_sp = ['CHARLESTON']
babette_sp = ['BABETE','BARBETTE']
logan_sp = ['LOG']
trix_sp = ['GRAN'] #Lorelai the first
michel_sp = ['MICHE','MICHELL']
tristin_sp = ['TRISTAN']
troubadour_sp = ['TROUBADOR']
tanna_sp = ['TANA'] #Rory's college roommate
prof_sp = ['PROFERSSOR']
brian_sp = ['BRAIN'] #band member
taylor_sp = ['TAYOR']

In [29]:
#replace incorrect names with correct names
DATA['Character'] = DATA['Character'].replace(lorelai_sp, value = 'LORELAI')
DATA['Character'] = DATA['Character'].replace(luke_sp, value = 'LUKE')
DATA['Character'] = DATA['Character'].replace(rory_sp, value = 'RORY')
DATA['Character'] = DATA['Character'].replace(richard_sp, value = 'RICHARD')
DATA['Character'] = DATA['Character'].replace(emily_sp, value = 'EMILY')
DATA['Character'] = DATA['Character'].replace(sookie_sp, value = 'SOOKIE')
DATA['Character'] = DATA['Character'].replace(caesar_sp, value = 'CAESAR')
DATA['Character'] = DATA['Character'].replace(christopher_sp, value = 'CHRISTOPHER')
DATA['Character'] = DATA['Character'].replace(lindsay_sp, value = 'LINDSAY')
DATA['Character'] = DATA['Character'].replace(headmaster_sp, value = 'HEADMASTER')
DATA['Character'] = DATA['Character'].replace(babette_sp, value = 'BABETTE')
DATA['Character'] = DATA['Character'].replace(logan_sp, value = 'LOGAN')
DATA['Character'] = DATA['Character'].replace(trix_sp, value = 'TRIX')
DATA['Character'] = DATA['Character'].replace(michel_sp, value = 'MICHEL')
DATA['Character'] = DATA['Character'].replace(tristin_sp, value = 'TRISTIN')
DATA['Character'] = DATA['Character'].replace(troubadour_sp, value = 'TROUBADOUR')
DATA['Character'] = DATA['Character'].replace(tanna_sp, value = 'TANNA')
DATA['Character'] = DATA['Character'].replace(prof_sp, value = 'PROFESSOR')
DATA['Character'] = DATA['Character'].replace(brian_sp, value = 'BRIAN')
DATA['Character'] = DATA['Character'].replace(taylor_sp, value = 'TAYLOR')

<ipython-input-29-a20c201ba9c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['Character'] = DATA['Character'].replace(lorelai_sp, value = 'LORELAI')
<ipython-input-29-a20c201ba9c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATA['Character'] = DATA['Character'].replace(luke_sp, value = 'LUKE')
<ipython-input-29-a20c201ba9c1>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [30]:
DATA['Character'].sort_values().unique()

array(['ADVISOR', 'AK', 'ALAN', 'ALEX', 'ALEXANDRA', 'ALISON', 'ALL',
       'ALTHEA', 'AMIR', 'ANDRE', 'ANDREA', 'ANDREW', 'ANDY', 'ANN',
       'ANNA', 'ANNABELLE', 'ANNOUNCER', 'ANTONIA', 'APRIL', 'ARLETTA',
       'ARTHUR', 'ASHER', 'ATTENDANT', 'AUBREY', 'AUCTIONEER', 'AUDREY',
       'AURORA', 'AVA', 'AYOR', 'BABETTE', 'BAND', 'BARBARA', 'BARRY',
       'BARTENDER', 'BEATRICE', 'BEAU', 'BELLBOY', 'BEN', 'BENNY', 'BERT',
       'BEVERLY', 'BIKER', 'BIKERS', 'BILL', 'BILLY', 'BOB', 'BOBBI',
       'BOBBY', 'BONNIE', 'BOOTSY', 'BOOZOO', 'BOTH', 'BOUNCER', 'BOY',
       'BOYS', 'BRAD', 'BRADLEY', 'BRANDON', 'BRENNON', 'BRIAN', 'BRIDE',
       'BRUCE', 'BRUNETTE', 'BUDDY', 'BUTCHER', 'CAESAR', 'CAITLYN',
       'CARL', 'CAROL', 'CAROLE', 'CAROLINE', 'CAROLYN', 'CARRIE',
       'CASHIER', 'CATHERINE', 'CELINE', 'CHAD', 'CHARLENE', 'CHARLIE',
       'CHARLOTTE', 'CHASE', 'CHEF', 'CHERRY', 'CHERYL', 'CHESTER',
       'CHILDREN', 'CHIP', 'CHRISSY', 'CHRISTINE', 'CHRISTOPHER',
       'CHRI

In [31]:
DATA[DATA['Character'] == 'TAYOR']

,Season,Episode,Title,time_raw,line_num,Character,lines


In [32]:
#extract year
DATA['Year'] = DATA.time_raw.str.split('-', n = 1, expand = True)[0]

In [33]:
DATA

,Season,Episode,Title,time_raw,line_num,Character,lines,Year
61758,01,01,The Pilot,2000-10-17T15:42:16Z,5,LORELAI,"Please, Luke. Please, please, please.",2000
61759,01,01,The Pilot,2000-10-17T15:42:16Z,6,LUKE,How many cups have you had this morning?,2000
61760,01,01,The Pilot,2000-10-17T15:42:16Z,7,LORELAI,None.,2000
61761,01,01,The Pilot,2000-10-17T15:42:16Z,8,LUKE,Plus...,2000
61762,01,01,The Pilot,2000-10-17T15:42:16Z,9,LORELAI,"Five, but yours is better.",2000
...,...,...,...,...,...,...,...,...
100865,07,22,Bon Voyage,2007-05-27T10:54:44Z,807,RORY,Well It wasn't very flattering.,2007
100866,07,22,Bon Voyage,2007-05-27T10:54:44Z,808,LORELAI,Well how will people know you're the world's ...,2007
100867,07,22,Bon Voyage,2007-05-27T10:54:44Z,809,RORY,I don't know.,2007
100868,07,22,Bon Voyage,2007-05-27T10:54:44Z,810,LORELAI,I guess they'll just have to read your stuff.,2007


# Create LIB

## LIB by Season

In [34]:
LIB = DATA.copy()

In [35]:
#add show title
LIB['Show'] = 'Gilmore Girls'

In [36]:
#subset data
LIB = LIB[['Show','Season','Episode','Title','Year','line_num','Character','lines']]

In [37]:
#count the number of epsiodes per season
LIB['Num_Episodes'] = LIB.groupby(['Season'])['Episode'].transform('nunique')
#count the number of lines per season
LIB['Num_Lines'] = LIB.groupby('Season')['Season'].transform('count')

In [38]:
#separate library
LIB2 = LIB.copy()

In [39]:
#subset to the number of seasons in the show
LIB = LIB.drop_duplicates(['Season'])

In [40]:
#subset data
LIB = LIB[['Show','Season','Num_Episodes','Year','Num_Lines']]

In [41]:
#reset index
LIB = LIB.reset_index(drop = True).set_index('Show')

In [42]:
LIB

,Season,Num_Episodes,Year,Num_Lines
Show,,,,
Gilmore Girls,01,21,2000,15687
Gilmore Girls,02,22,2001,18069
Gilmore Girls,03,22,2002,17407
Gilmore Girls,04,22,2003,17546
Gilmore Girls,05,22,2004,16510
Gilmore Girls,06,22,2005,15798
Gilmore Girls,07,22,2006,14908


## LIB by Episode

In [43]:
#count the number of lines per season and episode
LIB2['Num_Lines_per_Episode'] = LIB2.groupby(['Season','Episode'])['Season'].transform('count')

In [44]:
#unique episodes
LIB2 = LIB2.drop_duplicates(['Title'])

In [45]:
#subset the data
LIB2 = LIB2[['Show','Season','Num_Episodes','Episode','Title','Year','Num_Lines_per_Episode']]

In [46]:
#reset index
LIB2 = LIB2.reset_index(drop = True).set_index('Show')

In [47]:
#rename column
LIB2.rename(columns={"Num_Episodes":"Num_Episodes_per_Season"}, inplace=True)

In [48]:
LIB2

,Season,Num_Episodes_per_Season,Episode,Title,Year,Num_Lines_per_Episode
Show,,,,,,
Gilmore Girls,01,21,01,The Pilot,2000,610
Gilmore Girls,01,21,02,The Lorelai's First Day at Chilton,2000,586
Gilmore Girls,01,21,03,k*ll Me Now,2000,666
Gilmore Girls,01,21,04,The Deer Hunters,2000,708
Gilmore Girls,01,21,05,Rory's Birthday Parties,2000,745
...,...,...,...,...,...,...
Gilmore Girls,07,22,18,Hay Bale Maze,2007,745
Gilmore Girls,07,22,19,It's Just Like Riding A Bike,2007,788
Gilmore Girls,07,22,20,Lorelai? Lorelai?,2007,687


# Token Table

In [49]:
DATA.head()

,Season,Episode,Title,time_raw,line_num,Character,lines,Year
61758,01,01,The Pilot,2000-10-17T15:42:16Z,5,LORELAI,"Please, Luke. Please, please, please.",2000
61759,01,01,The Pilot,2000-10-17T15:42:16Z,6,LUKE,How many cups have you had this morning?,2000
61760,01,01,The Pilot,2000-10-17T15:42:16Z,7,LORELAI,None.,2000
61761,01,01,The Pilot,2000-10-17T15:42:16Z,8,LUKE,Plus...,2000
61762,01,01,The Pilot,2000-10-17T15:42:16Z,9,LORELAI,"Five, but yours is better.",2000


In [73]:
OHCO = ['Season','Episode','Character','line_num','sent_num','token_num']

In [74]:
SEAS = OHCO[:1]
EPI = OHCO[:2]
CHAR = OHCO[:3]
LINE = OHCO[:4]
SENT = OHCO[:5]
TOKEN = OHCO[:6]

In [75]:
GG = DATA[['Season','Episode','Character','line_num','lines']]

In [76]:
#GG['lines'] = GG['lines'].str.strip()

In [77]:
#SEASONS = GG.set_index(SEAS)
#EPISODES = GG.set_index(EPI)
#CHARACTERS = GG.set_index(CHAR)
CHARACTERS = GG.set_index(CHAR)
#SENT = GG.set_index(SENT)
#TOKENS2 = GG.set_index(TOKEN)

In [78]:
CHARACTERS

line_num  \
Season Episode Character             
01     01      LORELAI           5   
               LUKE              6   
               LORELAI           7   
               LUKE              8   
               LORELAI           9   
...                            ...   
07     22      RORY            807   
               LORELAI         808   
               RORY            809   
               LORELAI         810   
               RORY            811   

                                                                      lines  
Season Episode Character                                                     
01     01      LORELAI               Please, Luke. Please, please, please.   
               LUKE               How many cups have you had this morning?   
               LORELAI                                               None.   
               LUKE                                                Plus...   
               LORELAI                          Five, but yours is better.   
...                                                                     ...  
07     22      RORY                         Well It wasn't very flattering.  
               LORELAI     Well how will people know you're the world's ...  
               RORY                                           I don't know.  
               LORELAI        I guess they'll just have to read your stuff.  
               RORY                                             I guess so.  

[115925 rows x 2 columns]

## Split lines into sentences

In [79]:
#split lines into sentences
sent_pat = r'[.?!;:]+'
SENTS = CHARACTERS['lines'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = LINE

In [80]:
SENTS

sent_str
Season Episode Character line_num                                               
01     01      LORELAI   0                                          Please, Luke
                         1                                Please, please, please
                         2                                                      
               LUKE      0               How many cups have you had this morning
                         1                                                      
...                                                                          ...
07     22      RORY      1                                                      
               LORELAI   0          I guess they'll just have to read your stuff
                         1                                                      
               RORY      0                                            I guess so
                         1                                                      

[292915 rows x 1 columns]

In [ ]:
# SENTS = CHARACTERS.lines.apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
#         .stack()\
#         .to_frame('sent_str')
# SENTS.index.names = OHCO[:5]

In [81]:
# SENTS

In [82]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

/Users/alicebogdan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [83]:
SENTS

sent_str
Season Episode Character line_num                                                   
01     01      LORELAI   0                                              Please, Luke
                         1                                    Please, please, please
               LUKE      0                   How many cups have you had this morning
               LORELAI   0                                                      None
               LUKE      0                                                      Plus
...                                                                              ...
07     22      RORY      0                            Well It wasn't very flattering
               LORELAI   0         Well how will people know you're the world's g...
               RORY      0                                              I don't know
               LORELAI   0              I guess they'll just have to read your stuff
               RORY      0                                                I guess so

[176861 rows x 1 columns]

## Split sentences into tokens

In [ ]:
#split by spaces, single quotes, commas, and dashes
# token_pat = r"[\s',-]+"
# TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
#     .to_frame('token_str')
# TOKENS.index.names = SENT

In [84]:
#add "'" *apostrophe to remove cases like "lukes". Will convert to "luke" and "s"
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')
TOKENS.index.names = SENT

In [85]:
TOKENS.head(40)

token_str
Season Episode Character line_num sent_num          
01     01      LORELAI   0        0           Please
                                  1             Luke
                         1        0           Please
                                  1           please
                                  2           please
               LUKE      0        0              How
                                  1             many
                                  2             cups
                                  3             have
                                  4              you
                                  5              had
                                  6             this
                                  7          morning
               LORELAI   0        0             None
               LUKE      0        0             Plus
               LORELAI   0        0             Five
                                  1              but
                                  2            yours
                                  3               is
                                  4           better
               LUKE      0        0              You
                                  1             have
                                  2                a
                                  3          problem
               LORELAI   0        0              Yes
                                  1                I
                                  2               do
               LUKE      0        0           Junkie
               LORELAI   0        0            Angel
                         1        0              You
                                  1               ve
                                  2              got
                                  3            wings
                                  4             baby
               LUKE      0        0             Luke
                                  1                s
                         1        0             Yeah
               JOEY      0        0              You
                                  1             make
                                  2             that

In [86]:
TOKENS = TOKENS[TOKENS.token_str != '']

In [87]:
TOKENS

token_str
Season Episode Character line_num sent_num          
01     01      LORELAI   0        0           Please
                                  1             Luke
                         1        0           Please
                                  1           please
                                  2           please
...                                              ...
07     22      LORELAI   0        8             your
                                  9            stuff
               RORY      0        0                I
                                  1            guess
                                  2               so

[1144995 rows x 1 columns]

In [ ]:
TOKENS

## Extract Vocab

In [ ]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()

In [ ]:
TOKENS

In [ ]:
VOCAB = TOKENS.term_str.value_counts().to_frame('n')

In [ ]:
VOCAB.head(20)

In [ ]:
VOCAB.index.name = 'term_str'

Reference for variable names in VOCAB    
n:       count  
n_chars: len of term  
p:       probability  
s:       surpise  
i:       information  
h:       entropy  

In [ ]:
VOCAB['n_chars'] = VOCAB.index.str.len()
VOCAB['p'] = VOCAB['n'] / VOCAB['n'].sum()
VOCAB['s'] = 1 / VOCAB['p']
VOCAB['i'] = np.log2(VOCAB['s']) # Same as negative log probability (i.e. log likelihood)
VOCAB['h'] = VOCAB['p'] * VOCAB['i'] 
H = VOCAB['h'].sum()

In [ ]:
VOCAB

# Annotate VOCAB

## Get Max POS (Part-of-Space)

In [ ]:
VOCAB

## Add Stopwords

In [ ]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1